<a href="https://colab.research.google.com/github/vokulovskiy/Counting-people-at-the-gate/blob/main/%D0%9F%D0%BE%D0%B4%D1%81%D1%87%D0%B5%D1%82_%D0%BB%D1%8E%D0%B4%D0%B5%D0%B9_%D0%BD%D0%B0_%D0%BF%D1%80%D0%BE%D1%85%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9_%D0%BF%D0%BE_%D0%BF%D0%B5%D1%80%D0%B5%D1%81%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D1%8E_%D0%BB%D0%B8%D0%BD%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics
%pip install supervision
## Подключение гугл диска для сохранения модели
from google.colab import drive
drive.mount('/content/gdrive')
# Tестовое видео /content/gdrive/MyDrive/models/camera1_01.mp4

In [ ]:
#Подсчет людей пересекающих линию(мод)
import cv2

from ultralytics import YOLO
import supervision as sv
from supervision.notebook.utils import show_frame_in_notebook
import numpy as np
from google.colab.patches import cv2_imshow
from IPython import display
from supervision import VideoInfo, VideoSink

%matplotlib inline
display.clear_output()


# Задаем координаты начала и конца линии, которую мы будем использовать для подсчета проходящих объектов
#Camera1
#LINE_START = sv.Point(300,800)
#LINE_END = sv.Point(1000,600)
#Camera2
#LINE_START = sv.Point(220,740)
#LINE_END = sv.Point(690,700)
#Camera3
LINE_START = sv.Point(350,900)
LINE_END = sv.Point(900,750)

# Создаем объекты для работы с подсчетом объектов, аннотированием линии и аннотированием прямоугольных рамок вокруг объектов
line_counter = sv.LineZone(start=LINE_START, end=LINE_END)
line_annotator = sv.LineZoneAnnotator(thickness=2, text_thickness=1, text_scale=0.5)
box_annotator = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5
)
box_annotator_c = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5,
    color=sv.Color.white()
)
path='/content/gdrive/MyDrive/models/'
file_in='camera3_01.mp4'
file_out='camera3_01_mod.mp4'
video_info = VideoInfo.from_video_path(video_path=path+file_in)
# Создаем экземпляр модели YOLO
model = YOLO("yolov8m.pt")
kadr = 0

#Запись результата 
with VideoSink(target_path=path+file_out, video_info=video_info) as s:
    # Итерируемся по кадрам видео
    for result in model.track(source=path+file_in, show=True, stream=True, agnostic_nms=True):
        
        # Получаем текущий кадр и результаты детекции
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)
        kadr += 1

        # Присваиваем каждому объекту ID, чтобы отслеживать его в последующих кадрах
        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)
        
        # Оставляем только объекты, соответствующие нужным классам
        detections = detections[(detections.class_id == 0)]

        # Формируем метки для объектов
        labels = [
            f"{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]

        # Аннотируем прямоугольными рамками объекты на кадре
        frame = box_annotator.annotate(
            scene=frame, 
            detections=detections,
            labels=labels
        )
        #Умеьшаем рамки для детектирования в n раз
        n=10
        if result.boxes.id is not None:
            for i in range(len(detections.xyxy)):
                x1=detections.xyxy[i][0]
                y1=detections.xyxy[i][1]
                x2=detections.xyxy[i][2]
                y2=detections.xyxy[i][3]
                detections.xyxy[i][0]=x1+(x2-x1)/2-(x2-x1)/n
                detections.xyxy[i][1]=y1+(y2-y1)/2-(y2-y1)/n
                detections.xyxy[i][2]=x1+(x2-x1)/2+(x2-x1)/n
                detections.xyxy[i][3]=y1+(y2-y1)/2+(y2-y1)/n

        # Рисуем прямоугольные рамки центров объектов на кадре
        frame = box_annotator_c.annotate(
            scene=frame, 
            detections=detections,
            labels=None, skip_label=True
        )
        # Подсчитываем количество объектов, пересекающих линию
        line_counter.trigger(detections=detections)
        
        # Аннотируем линию на кадре
        line_annotator.annotate(frame=frame, line_counter=line_counter)

        # Отображаем кадр
        s.write_frame(frame=frame)
        #display.clear_output()
        #show_frame_in_notebook(frame, (16, 16))
        #cv2_imshow(frame)
        #cv2.imshow("yolov8", frame)

        # Выходим из цикла, если пользователь нажал клавишу ESC
        if (cv2.waitKey(30) == 27):
            break


In [ ]:
Результаты
1 проходная: https://drive.google.com/file/d/1-3zJb_yzwqpcIzdx7OfAKuN-0LRAWbwi/view?usp=sharing

2 проходная: https://drive.google.com/file/d/1-8-JoQWtt1DbDlIXgvFYfTs4nB8HVijI/view?usp=sharing

3 проходная: https://drive.google.com/file/d/1-C9ymEFjJqTxyulFnQ5yj2B2PJ6311sG/view?usp=sharing